In [1]:
from skimage import feature
import cv2
import numpy as np
import os
from skimage import feature
import matplotlib.pyplot as plt
from PIL import Image

In [2]:

root_folder = "preprocessed_data_"

# Đường dẫn thư mục đầu ra
output_folder = "final_image_"
def preprocess():
    for subdir, _, files in os.walk(root_folder):
        for file in files:
            file_path = os.path.join(subdir, file)
            img = cv2.imread(file_path)

            height, width, _ = img.shape

            # Kích thước cắt
            crop_width, crop_height = 480, 480

            # Tính toán tọa độ bắt đầu để có được vùng trung tâm
            start_x = (width - crop_width) // 2
            start_y = (height - crop_height) // 2

            # Cắt ảnh
            cropped_img = img[start_y:start_y+crop_height, start_x:start_x+crop_width]
            # Làm mờ ảnh
            blurred = cv2.GaussianBlur(cropped_img, (5, 5), 0)

            # Phát hiện cạnh
            edges = cv2.Canny(blurred, 750, 800)

            # Loại bỏ các cạnh không mong muốn
            kernel = np.ones((3, 3), np.uint8)
            edges = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)

            # Tạo đường dẫn đến thư mục đầu ra
            output_subdir = os.path.relpath(subdir, root_folder)
            output_subfolder = os.path.join(output_folder, output_subdir)

            # Tạo thư mục con trong thư mục đầu ra nếu chưa tồn tại
            os.makedirs(output_subfolder, exist_ok=True)

            # Tạo đường dẫn đến tệp tin đầu ra
            output_path = os.path.join(output_subfolder, file)

            # Lưu ảnh đã xử lý
            cv2.imwrite(output_path, edges)

    print("Processing complete.")
# preprocess()

In [3]:
def extract_lbp_features(img):
    if len(img.shape) == 3:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        gray = img.copy()
    
    # Thực hiện LBP
    lbp = feature.local_binary_pattern(gray, P=8, R=1, method="uniform")

    # Tính histogram của các giá trị LBP
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 10), range=(0, 10))

    # Chuẩn hóa histogram
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-7)

    return hist

In [4]:
root_folder = "final_image_"

# Dữ liệu features và nhãn
features = []
labels = []

In [5]:

features = []
labels = []

# Duyệt qua từng thư mục con
for subdir, _, files in os.walk(root_folder):
    # Bỏ qua thư mục chính
    if subdir == root_folder:
        continue

    label = os.path.basename(subdir)  # Nhãn của thư mục con
    object_features = []  # Danh sách đặc trưng của đối tượng

    # Duyệt qua từng tệp tin ảnh trong thư mục con
    for file in files:
        # Đường dẫn đầy đủ đến tệp tin ảnh
        file_path = os.path.join(subdir, file)

        # Đọc ảnh
        img = cv2.imread(file_path)

        # Trích xuất đặc trưng từ ảnh và thêm vào danh sách đặc trưng của đối tượng
        img_features = extract_lbp_features(img)
        object_features.append(img_features)

    # Gộp các đặc trưng của đối tượng thành một giá trị duy nhất (ví dụ: trung bình)
    object_features_mean = np.mean(object_features, axis=0)

    # Thêm đặc trưng và nhãn của đối tượng vào danh sách chung
    features.append(object_features_mean)
    labels.append(label)


In [6]:
print(len(labels))

276


In [7]:
import preprocess_

In [8]:
def preprocess_image(input_path):
    image = Image.open(input_path)
    image = np.array(image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    processed_image = preprocess_.process(image)
    return processed_image
# test = preprocess_image(test_path)


In [9]:
database_features = features
labels = labels

In [13]:

def test(input_path,threshold):
    image = Image.open(input_path)
    image = np.array(image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    test = preprocess_.process(image)
    
    height, width = test.shape

    # Kích thước cắt
    crop_width, crop_height = 480, 480

    start_x = (width - crop_width) // 2
    start_y = (height - crop_height) // 2

    # Cắt ảnh
    cropped_img = test[start_y:start_y+crop_height, start_x:start_x+crop_width]
    # # Làm mờ ảnh
    blurred = cv2.GaussianBlur(cropped_img, (5, 5), 0)

    # Phát hiện cạnh
    edges = cv2.Canny(blurred, 750, 800)

    kernel = np.ones((3, 3), np.uint8)
    edges_ = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)
    img_features = extract_lbp_features(edges_)

    distances = np.linalg.norm(database_features - img_features, axis=1)

    min_distance_index = np.argmin(distances)
    if distances[min_distance_index] < threshold:
        predicted_label = labels[min_distance_index]
        print(f"The image is predicted to be of the object: {predicted_label}")
    else:
        print("No match found.")


In [18]:
test("image_test/027_L4.png",0.1)

The image is predicted to be of the object: 027_L
